<a href="https://colab.research.google.com/github/pixelfreund/pixelfreund/blob/main/Recipe_Transformation_Claude_API_Batch_Request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 1. Setup & Installation

In [1]:
# Benötigte Pakete installieren
!pip install anthropic beautifulsoup4 --quiet

print("✅ Pakete installiert")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 8.4 MB/s eta 0:00:00
✅ Pakete installiert


In [2]:
# Google Drive mounten
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive verbunden")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive verbunden


In [3]:
# API-Key eingeben (wird nicht gespeichert)
from getpass import getpass
import os

ANTHROPIC_API_KEY = getpass("🔑 Anthropic API Key eingeben: ")
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

print("✅ API-Key gesetzt")

🔑 Anthropic API Key eingeben: ··········
✅ API-Key gesetzt


In [4]:
# Konfiguration
from pathlib import Path

# Pfade
INPUT_DIR = Path("/content/drive/MyDrive/1000-python/recipe-transformer/files")
OUTPUT_DIR = Path("/content/drive/MyDrive/1000-python/recipe-transformer/outputs")

# Output-Verzeichnis erstellen
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Dateien zählen
html_files = list(INPUT_DIR.glob("*.html")) + list(INPUT_DIR.glob("*.htm"))
print(f"📁 Input-Verzeichnis: {INPUT_DIR}")
print(f"📁 Output-Verzeichnis: {OUTPUT_DIR}")
print(f"📄 Gefundene HTML-Dateien: {len(html_files)}")

📁 Input-Verzeichnis: /content/drive/MyDrive/1000-python/recipe-transformer/files
📁 Output-Verzeichnis: /content/drive/MyDrive/1000-python/recipe-transformer/outputs
📄 Gefundene HTML-Dateien: 446


In [5]:
# Konfiguration
from pathlib import Path

# Pfade
INPUT_DIR = Path("/content/drive/MyDrive/1000-python/recipe-transformer/files")
OUTPUT_DIR = Path("/content/drive/MyDrive/1000-python/recipe-transformer/outputs")

# Output-Verzeichnis erstellen
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Dateien zählen
html_files = list(INPUT_DIR.glob("*.html")) + list(INPUT_DIR.glob("*.htm"))
print(f"📁 Input-Verzeichnis: {INPUT_DIR}")
print(f"📁 Output-Verzeichnis: {OUTPUT_DIR}")
print(f"📄 Gefundene HTML-Dateien: {len(html_files)}")

📁 Input-Verzeichnis: /content/drive/MyDrive/1000-python/recipe-transformer/files
📁 Output-Verzeichnis: /content/drive/MyDrive/1000-python/recipe-transformer/outputs
📄 Gefundene HTML-Dateien: 446


In [7]:
# System-Prompt gemäss Recipe-Skill
SYSTEM_PROMPT = """Schweizer Food-Blogger für foodies.pixelfreund.ch. Transformiere Rezepte ins Standard-Format.

## Format

### Struktur (in dieser Reihenfolge)
1. **Titel** (H1) - kann Zusatz haben
2. **Einleitung** (2-4 Sätze) - Herkunft, Besonderheit, persönliche Note
3. **Zutaten** (H2) - mit H3-Kategorien bei Bedarf
4. **Zubereitung** (H2) - H3-Phasen, Schritte als Liste
5. **Serviervorschlag** (1 Zeile)
6. **Bilder** (nur "## Bilder")
7. **Quellen** (falls vorhanden)

### Stil
- Schweizer Rechtschreibung: "ss" statt "ß"
- Imperativ: "Butter schmelzen"
- Mengen dezimal: 300.0 ml, 2.0 EL, 1.5 TL
- Temperatur: 200°C
- Nur "-", kein "–"
- Einheiten: g, ml, EL, TL, Stück, Scheiben
- Persönlich, nicht übertrieben

### Beispiele

**Einleitung:** "Der Croque Monsieur ist der König unter den französischen Sandwiches - ein Klassiker der Bistroküche, bei dem knuspriges Brot auf cremige Füllung trifft."

**Zutaten:**
## Zutaten
### Für die Sauce
- 300.0 ml Milch
- 30.0 g Butter

**Zubereitung:**
## Zubereitung
### Béchamelsauce
- Butter schmelzen
- Mehl einrühren

Antworte NUR mit formatiertem Markdown-Rezept."""

print(f"📝 System-Prompt geladen ({len(SYSTEM_PROMPT)} Zeichen)")

📝 System-Prompt geladen (1103 Zeichen)


In [8]:
from bs4 import BeautifulSoup
import re

def fix_encoding(text):
    """Korrigiert häufige UTF-8 Encoding-Probleme."""
    if not text:
        return ""

    replacements = {
        'Ã¤': 'ä', 'Ã¶': 'ö', 'Ã¼': 'ü',
        'Ã„': 'Ä', 'Ã–': 'Ö', 'Ãœ': 'Ü',
        'ÃŸ': 'ss', 'ß': 'ss',  # Direkt zu Schweizer Schreibweise
        'Ã©': 'é', 'Ã¨': 'è', 'Ã ': 'à',
        'Ã¯': 'ï', 'Ã®': 'î', 'Ã´': 'ô',
        'Ä‡': 'ć', 'Ä†': 'Ć', 'Ä': 'č',
        'Â ': ' ', 'Â': '',
        '&amp;': '&', '&lt;': '<', '&gt;': '>',
        '&nbsp;': ' ',
    }

    for old, new in replacements.items():
        text = text.replace(old, new)

    return text

def extract_text_from_html(html_content):
    """Extrahiert lesbaren Text aus HTML."""
    html_content = fix_encoding(html_content)
    soup = BeautifulSoup(html_content, 'html.parser')

    # Script und Style entfernen
    for tag in soup(['script', 'style', 'nav', 'footer', 'header']):
        tag.decompose()

    # Titel extrahieren
    title = ""
    title_tag = soup.find('h1') or soup.find('title')
    if title_tag:
        title = title_tag.get_text(strip=True)

    # Text extrahieren
    text = soup.get_text(separator='\n', strip=True)
    text = re.sub(r'\n{3,}', '\n\n', text)

    return title, text

def load_recipe(filepath):
    """Lädt eine HTML-Datei und extrahiert den Inhalt."""
    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']

    for encoding in encodings:
        try:
            with open(filepath, 'r', encoding=encoding) as f:
                content = f.read()
            title, text = extract_text_from_html(content)
            return {
                'filename': filepath.stem,
                'title': title or filepath.stem,
                'content': text,
                'html': content
            }
        except UnicodeDecodeError:
            continue

    raise ValueError(f"Konnte {filepath} nicht lesen")

# Test mit einer Datei
if html_files:
    test_recipe = load_recipe(html_files[0])
    print(f"✅ Test erfolgreich: {test_recipe['title'][:50]}...")
    print(f"   Inhalt: {len(test_recipe['content'])} Zeichen")

✅ Test erfolgreich: Aargauer Kartoffelwähe...
   Inhalt: 1087 Zeichen


In [9]:
import json

def create_batch_request(recipe, index):
    """Erstellt einen einzelnen Batch-Request gemäss Anthropic API Spezifikation.

    Struktur gemäss https://platform.claude.com/docs/en/build-with-claude/batch-processing:
    - custom_id: Eindeutige ID für den Request
    - params: Standard Messages API Parameter
    """

    user_message = f"""Standardisiere folgendes Rezept:

**Titel:** {recipe['title']}

**Originaler Inhalt:**
{recipe['content'][:8000]}
"""

    # Sichere custom_id erstellen (nur alphanumerisch + Bindestrich/Unterstrich)
    safe_filename = re.sub(r'[^a-zA-Z0-9_-]', '_', recipe['filename'])[:50]

    return {
        "custom_id": f"recipe-{index:04d}-{safe_filename}",
        "params": {
            "model": "claude-sonnet-4-5-20250929",
            "max_tokens": 4096,
            "system": SYSTEM_PROMPT,
            "messages": [
                {"role": "user", "content": user_message}
            ]
        }
    }

# Alle Rezepte laden und Batch-Requests erstellen
print("📖 Lade Rezepte...")
recipes = []
batch_requests = []
errors = []

for i, filepath in enumerate(html_files):
    try:
        recipe = load_recipe(filepath)
        recipes.append(recipe)
        batch_requests.append(create_batch_request(recipe, i))

        if (i + 1) % 50 == 0:
            print(f"   {i + 1}/{len(html_files)} geladen...")
    except Exception as e:
        errors.append((filepath.name, str(e)))

print(f"\n✅ {len(recipes)} Rezepte geladen")
print(f"✅ {len(batch_requests)} Batch-Requests erstellt")
if errors:
    print(f"⚠️  {len(errors)} Fehler:")
    for name, err in errors[:5]:
        print(f"   - {name}: {err}")

📖 Lade Rezepte...
   50/446 geladen...
   100/446 geladen...
   150/446 geladen...
   200/446 geladen...
   250/446 geladen...
   300/446 geladen...
   350/446 geladen...
   400/446 geladen...

✅ 446 Rezepte geladen
✅ 446 Batch-Requests erstellt


In [10]:
# Mapping für spätere Zuordnung speichern
id_to_filename = {}
for i, recipe in enumerate(recipes):
    safe_filename = re.sub(r'[^a-zA-Z0-9_-]', '_', recipe['filename'])[:50]
    custom_id = f"recipe-{i:04d}-{safe_filename}"
    id_to_filename[custom_id] = recipe['filename']

print(f"📋 Mapping erstellt: {len(id_to_filename)} Einträge")

# Beispiel anzeigen
if batch_requests:
    print("\n📝 Beispiel-Request:")
    example = batch_requests[0]
    print(f"   custom_id: {example['custom_id']}")
    print(f"   model: {example['params']['model']}")
    print(f"   max_tokens: {example['params']['max_tokens']}")
    print(f"   message length: {len(example['params']['messages'][0]['content'])} chars")

📋 Mapping erstellt: 446 Einträge

📝 Beispiel-Request:
   custom_id: recipe-0000-Aargauer_Kartoffelwa_he
   model: claude-sonnet-4-5-20250929
   max_tokens: 4096
   message length: 1180 chars


In [33]:
import anthropic

# Client initialisieren
client = anthropic.Anthropic()

print(f"🚀 Sende Batch mit {len(batch_requests)} Requests...")
print("   Endpoint: POST /v1/messages/batches")
print("   (Dies kann einige Sekunden dauern)\n")

# Batch erstellen - gemäss API Dokumentation
batch = client.messages.batches.create(requests=batch_requests)

print(f"✅ Batch erstellt!")
print(f"   ID: {batch.id}")
print(f"   Status: {batch.processing_status}")
print(f"   Erstellt: {batch.created_at}")
print(f"   Läuft ab: {batch.expires_at}")
print(f"\n   Request Counts:")
print(f"   - Processing: {batch.request_counts.processing}")
print(f"   - Succeeded: {batch.request_counts.succeeded}")
print(f"   - Errored: {batch.request_counts.errored}")

# Batch-ID speichern für spätere Verwendung
BATCH_ID = batch.id
print(f"\n💾 Batch-ID für spätere Verwendung: {BATCH_ID}")

🚀 Sende Batch mit 446 Requests...
   Endpoint: POST /v1/messages/batches
   (Dies kann einige Sekunden dauern)

✅ Batch erstellt!
   ID: msgbatch_013knYFzi3hhNsmdw4mPERzf
   Status: in_progress
   Erstellt: 2025-12-29 14:10:05.706755+00:00
   Läuft ab: 2025-12-30 14:10:05.706755+00:00

   Request Counts:
   - Processing: 446
   - Succeeded: 0
   - Errored: 0

💾 Batch-ID für spätere Verwendung: msgbatch_013knYFzi3hhNsmdw4mPERzf


In [35]:
import time
from datetime import datetime
from IPython.display import clear_output

def format_time(seconds):
    """Formatiert Sekunden als HH:MM:SS."""
    hours, remainder = divmod(int(seconds), 3600)
    mins, secs = divmod(remainder, 60)
    if hours > 0:
        return f"{hours:02d}:{mins:02d}:{secs:02d}"
    return f"{mins:02d}:{secs:02d}"

def poll_batch_status(batch_id, interval=60):
    """
    Überwacht den Batch-Status bis zur Fertigstellung.

    Gemäss API Dokumentation:
    - processing_status: 'in_progress' -> 'ended'
    - request_counts zeigt Fortschritt
    """

    start_time = time.time()

    while True:
        # Status abrufen via GET /v1/messages/batches/{batch_id}
        batch = client.messages.batches.retrieve(batch_id)
        elapsed = time.time() - start_time

        # Status anzeigen
        clear_output(wait=True)

        counts = batch.request_counts
        total = counts.processing + counts.succeeded + counts.errored + counts.canceled + counts.expired
        done = counts.succeeded + counts.errored + counts.canceled + counts.expired

        progress = (done / total * 100) if total > 0 else 0

        print("🔄 BATCH-STATUS")
        print("=" * 50)
        print(f"Batch-ID: {batch_id}")
        print(f"Status: {batch.processing_status}")
        print(f"Laufzeit: {format_time(elapsed)}")
        print()
        print(f"Fortschritt: [{('█' * int(progress/5)):20s}] {progress:.1f}%")
        print()
        print(f"   ✅ Erfolgreich: {counts.succeeded}")
        print(f"   ⏳ In Bearbeitung: {counts.processing}")
        print(f"   ❌ Fehler: {counts.errored}")
        print(f"   🚫 Abgebrochen: {counts.canceled}")
        print(f"   ⏰ Abgelaufen: {counts.expired}")
        print("=" * 50)

        # Prüfen ob Batch fertig ist
        if batch.processing_status == 'ended':
            print(f"\n🎉 Batch abgeschlossen nach {format_time(elapsed)}!")
            if batch.results_url:
                print(f"📥 Ergebnisse verfügbar: {batch.results_url}")
            return batch

        print(f"\nNächste Aktualisierung in {interval} Sekunden...")
        print("(Du kannst diese Zelle unterbrechen und später fortfahren)")
        time.sleep(interval)

# Status überwachen (60 Sekunden Intervall)
print("Starte Status-Überwachung...\n")

final_batch = poll_batch_status(BATCH_ID, interval=60)

🔄 BATCH-STATUS
Batch-ID: msgbatch_013knYFzi3hhNsmdw4mPERzf
Status: ended
Laufzeit: 02:00

Fortschritt: [████████████████████] 100.0%

   ✅ Erfolgreich: 446
   ⏳ In Bearbeitung: 0
   ❌ Fehler: 0
   🚫 Abgebrochen: 0
   ⏰ Abgelaufen: 0

🎉 Batch abgeschlossen nach 02:00!
📥 Ergebnisse verfügbar: https://api.anthropic.com/v1/messages/batches/msgbatch_013knYFzi3hhNsmdw4mPERzf/results


In [37]:
# Falls nötig: Batch-ID manuell eingeben
# BATCH_ID = "msgbatch_xxxxx"  # Hier deine Batch-ID einfügen

# Einzelner Status-Check
batch = client.messages.batches.retrieve(BATCH_ID)

print(f"Batch-ID: {BATCH_ID}")
print(f"Status: {batch.processing_status}")
print(f"\nRequest Counts:")
print(f"   Erfolgreich: {batch.request_counts.succeeded}")
print(f"   In Bearbeitung: {batch.request_counts.processing}")
print(f"   Fehler: {batch.request_counts.errored}")
print(f"   Abgebrochen: {batch.request_counts.canceled}")
print(f"   Abgelaufen: {batch.request_counts.expired}")

if batch.results_url:
    print(f"\n📥 Results URL: {batch.results_url}")

Batch-ID: msgbatch_013knYFzi3hhNsmdw4mPERzf
Status: ended

Request Counts:
   Erfolgreich: 446
   In Bearbeitung: 0
   Fehler: 0
   Abgebrochen: 0
   Abgelaufen: 0

📥 Results URL: https://api.anthropic.com/v1/messages/batches/msgbatch_013knYFzi3hhNsmdw4mPERzf/results


In [38]:
def download_and_save_results(batch_id, output_dir, id_mapping):
    """
    Lädt Ergebnisse und speichert sie als Markdown-Dateien.

    Gemäss API Dokumentation:
    - Streaming via client.messages.batches.results()
    - Result types: succeeded, errored, canceled, expired
    - custom_id für Zuordnung verwenden (Reihenfolge nicht garantiert)
    """

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    success_count = 0
    error_count = 0
    error_details = []

    print("📥 Lade Ergebnisse (Streaming)...")

    # Ergebnisse streamen - gemäss API Dokumentation
    for result in client.messages.batches.results(batch_id):
        custom_id = result.custom_id

        # Original-Dateiname aus Mapping holen
        original_filename = id_mapping.get(custom_id, custom_id)

        if result.result.type == 'succeeded':
            # Markdown-Inhalt aus der Message extrahieren
            message = result.result.message
            content = message.content[0].text

            # Als Markdown speichern
            output_file = output_dir / f"{original_filename}.md"
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(content)

            success_count += 1

            if success_count % 25 == 0:
                print(f"   {success_count} Dateien gespeichert...")

        elif result.result.type == 'errored':
            error_count += 1
            error_details.append({
                'custom_id': custom_id,
                'filename': original_filename,
                'error_type': result.result.error.type if hasattr(result.result, 'error') else 'unknown',
                'error': str(result.result)
            })

        elif result.result.type == 'canceled':
            error_count += 1
            error_details.append({
                'custom_id': custom_id,
                'filename': original_filename,
                'error_type': 'canceled',
                'error': 'Request was canceled'
            })

        elif result.result.type == 'expired':
            error_count += 1
            error_details.append({
                'custom_id': custom_id,
                'filename': original_filename,
                'error_type': 'expired',
                'error': 'Request expired (24h limit)'
            })

    print(f"\n✅ Download abgeschlossen!")
    print(f"   Erfolgreich: {success_count}")
    print(f"   Fehler: {error_count}")
    print(f"   Gespeichert in: {output_dir}")

    return success_count, error_count, error_details

# Ergebnisse herunterladen und speichern
success, errors, error_list = download_and_save_results(
    BATCH_ID,
    OUTPUT_DIR,
    id_to_filename
)

📥 Lade Ergebnisse (Streaming)...
   25 Dateien gespeichert...
   50 Dateien gespeichert...
   75 Dateien gespeichert...
   100 Dateien gespeichert...
   125 Dateien gespeichert...
   150 Dateien gespeichert...
   175 Dateien gespeichert...
   200 Dateien gespeichert...
   225 Dateien gespeichert...
   250 Dateien gespeichert...
   275 Dateien gespeichert...
   300 Dateien gespeichert...
   325 Dateien gespeichert...
   350 Dateien gespeichert...
   375 Dateien gespeichert...
   400 Dateien gespeichert...
   425 Dateien gespeichert...

✅ Download abgeschlossen!
   Erfolgreich: 446
   Fehler: 0
   Gespeichert in: /content/drive/MyDrive/1000-python/recipe-transformer/outputs


In [39]:
# Gespeicherte Dateien auflisten
output_files = list(OUTPUT_DIR.glob("*.md"))
print(f"📁 {len(output_files)} Markdown-Dateien erstellt")
print()

# Beispiel anzeigen
if output_files:
    sample_file = output_files[0]
    print(f"📝 Beispiel: {sample_file.name}")
    print("=" * 50)
    with open(sample_file, 'r', encoding='utf-8') as f:
        content = f.read()
    print(content[:2000])
    if len(content) > 2000:
        print("\n[... gekürzt ...]")

📁 446 Markdown-Dateien erstellt

📝 Beispiel: Aargauer Kartoffelwähe.md
# Aargauer Kartoffelwähe

Die Aargauer Kartoffelwähe ist ein herzhafter Klassiker aus der Schweizer Wähenküche. Mit knusprigem Speck, Zwiebeln und geriebenen Kartoffeln in cremigem Rahm vereint sie alles, was eine deftige Wähe ausmacht. Ein Stück Schweizer Tradition, das besonders an kühlen Tagen für wohlige Wärme sorgt.

## Zutaten

### Für den Kuchenteig
- 150.0 g Mehl
- 0.25 TL Salz
- 50.0 g Butter, in Stücken, kalt
- 3.0 EL Wasser

### Für den Belag
- 200.0 g Speckwürfeli
- 2.0 Stück grosse Zwiebeln (ca. 250 g), fein gehackt
- 600.0 g fest kochende Kartoffeln, grob gerieben
- 2.0 Stück Eier
- 250.0 ml Vollrahm
- 1.0 TL Salz
- Muskat
- Pfeffer

## Zubereitung

### Teig
- Mehl und Salz in einer Schüssel mischen
- Butter beigeben und von Hand zu einer gleichmässig krümeligen Masse verreiben
- Wasser beigeben und rasch zu einem weichen Teig zusammenfügen, nicht kneten
- Teig flach drücken, zugedeckt ca. 30 Min. küh

In [40]:
# Statistiken
print("📊 ZUSAMMENFASSUNG")
print("=" * 50)
print(f"Input-Dateien (HTML):  {len(html_files)}")
print(f"Output-Dateien (MD):   {len(output_files)}")
print(f"Erfolgsquote:          {len(output_files)/len(html_files)*100:.1f}%")
print()
print(f"📁 Output-Verzeichnis:")
print(f"   {OUTPUT_DIR}")
print()
print("💰 KOSTEN (geschätzt):")
print(f"   Batch-Rabatt: 50%")
print(f"   ~$1.50/M Input, ~$7.50/M Output (Sonnet 4.5)")

📊 ZUSAMMENFASSUNG
Input-Dateien (HTML):  446
Output-Dateien (MD):   446
Erfolgsquote:          100.0%

📁 Output-Verzeichnis:
   /content/drive/MyDrive/1000-python/recipe-transformer/outputs

💰 KOSTEN (geschätzt):
   Batch-Rabatt: 50%
   ~$1.50/M Input, ~$7.50/M Output (Sonnet 4.5)
